# 🚀 使用 Hugging Face PEFT 库实现 LoRA 微调 - 完整教程

## 📖 教程概述

本教程将展示如何使用 **Hugging Face PEFT（Parameter-Efficient Fine-Tuning）** 库实现 LoRA 微调，这是工业界推荐的标准方法。

### 🎯 为什么使用 PEFT 库？

#### ✅ PEFT 库的优势

1. **工业级实现**：经过大规模验证，稳定可靠
2. **易于使用**：几行代码即可应用 LoRA
3. **功能丰富**：支持 LoRA、LoHa、LoKr 等多种方法
4. **生态集成**：与 Transformers、Accelerate 无缝集成
5. **持续维护**：Hugging Face 官方支持

#### 📊 与手动实现的对比

| 特性 | 手动实现 | PEFT 库 |
|------|---------|---------|
| **代码量** | 多（需实现 LoRA 层） | 少（几行配置） |
| **学习价值** | 深入理解原理 | 快速应用实践 |
| **维护成本** | 高（需自己维护） | 低（官方维护） |
| **功能完整性** | 基础功能 | 丰富功能 |
| **错误风险** | 较高 | 较低 |
| **适用场景** | 学习研究 | 生产环境 |

### 🎓 学习路径建议

1. **第一步**：学习手动实现（理解原理）
2. **第二步**：学习 PEFT 库（掌握工具）
3. **第三步**：实际项目中使用 PEFT

---

## 📦 核心依赖说明

本教程使用以下经过测试的依赖版本组合：

- **torch==2.8.0**：PyTorch 深度学习框架
- **transformers==4.51.3**：Hugging Face 模型库
- **peft==0.13.2**：参数高效微调库（核心）
- **accelerate==1.0.1**：分布式训练加速库
- **pandas==2.2.2**：数据处理
- **numpy==2.0.2**：数值计算

💡 **注意**：这些版本已经过兼容性测试，建议使用相同版本。


# 1️⃣ 环境准备和依赖安装

## 📦 一键安装所有依赖

以下命令将安装所有必需的依赖包。我们使用 `-q` 参数来减少输出信息。

### 🔍 依赖包说明

- **torch**：PyTorch 深度学习框架（支持 CUDA 加速）
- **transformers**：Hugging Face 的预训练模型库
- **peft**：参数高效微调库（本教程的核心）
- **accelerate**：分布式训练加速库
- **pandas/numpy**：数据处理和数值计算
- **tqdm**：进度条显示
- **matplotlib**：数据可视化


In [ ]:
# 📦 安装依赖包
# 说明：使用 -q 参数安静模式安装，--upgrade 确保安装最新版本
# 如果在 Colab 中运行，某些包可能已预装，会自动跳过

%pip install -q --upgrade torch==2.5.1 --index-url https://download.pytorch.org/whl/cu121 transformers==4.51.3 peft==0.13.2 accelerate==1.0.1 pandas==2.2.2 numpy==2.0.2 tqdm==4.67.1 matplotlib==3.10.7 requests==2.32.5 safetensors==0.6.2

print("✅ 依赖包安装完成！")
print("💡 建议重启 Runtime 以确保新包正常工作（Runtime -> Restart Runtime）")


# 2️⃣ 导入依赖库和环境检查

## 🔧 核心库导入说明

下面我们将导入所有需要的库，并检查运行环境。


In [ ]:
# 🔧 导入所有必需的库
# 功能：准备训练所需的所有依赖

import io
import os
import random
import zipfile

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForSequenceClassification,  # 序列分类模型（用于文本分类）
    AutoTokenizer,                        # 自动选择合适的分词器
    get_linear_schedule_with_warmup,     # 带预热的线性学习率调度器
)
from peft import (
    LoraConfig,        # LoRA 配置类
    TaskType,          # 任务类型枚举
    get_peft_model,    # 将 LoRA 应用到模型的核心函数
)
from tqdm import tqdm  # 进度条显示
import requests        # HTTP 请求（用于下载数据）

# 🎯 设置随机种子，确保结果可复现
# 说明：在深度学习中，很多操作涉及随机性（如参数初始化、数据打乱）
#      固定随机种子可以让每次运行得到相同的结果，便于调试和对比
torch.manual_seed(42)   # PyTorch 的随机种子
random.seed(42)         # Python 标准库的随机种子

# 🎮 设备选择：优先使用 GPU，如果没有则使用 CPU
# 说明：CUDA 是 NVIDIA GPU 的并行计算平台
#      torch.cuda.is_available() 检查是否有可用的 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 📊 显示环境信息
print("🔍 环境检查")
print("=" * 60)
print(f"🎮 使用设备: {device}")
if torch.cuda.is_available():
    print(f"   GPU 型号: {torch.cuda.get_device_name(0)}")
    print(f"   GPU 数量: {torch.cuda.device_count()}")
    print(f"   CUDA 版本: {torch.version.cuda}")
else:
    print("   ⚠️  未检测到 GPU，将使用 CPU（训练速度较慢）")

print(f"🐍 Python 库版本:")
print(f"   PyTorch: {torch.__version__}")
try:
    import transformers
    print(f"   Transformers: {transformers.__version__}")
    import peft
    print(f"   PEFT: {peft.__version__}")
except ImportError as e:
    print(f"   ⚠️ 部分库未安装: {e}")

print("\n✅ 环境准备完成！")


# 3️⃣ 数据准备和预处理

## 📊 数据集介绍

我们使用 **SMS Spam Collection** 数据集进行垃圾短信分类任务：

- **数据来源**：UCI Machine Learning Repository
- **数据规模**：约 5,572 条短信
- **任务类型**：二分类（ham 正常短信 vs spam 垃圾短信）
- **标签分布**：不平衡（ham 约 87%, spam 约 13%）

## 🔧 数据处理流程

```
原始数据下载 → 类别平衡采样 → 划分训练/验证/测试集 → 分词编码 → DataLoader加载
```


In [ ]:
# 📊 数据处理工具函数
# 功能：提供数据下载、平衡、划分和加载的完整工具集

def create_balanced_dataset(df):
    """
    创建类别平衡的数据集
    
    🎯 目的：解决数据不平衡问题
    原始数据中 ham（正常短信）远多于 spam（垃圾短信），
    这会导致模型偏向预测多数类。通过下采样多数类，
    使两个类别的样本数量相等。
    
    参数：
        df: pandas DataFrame，包含 Label 和 Text 列
        
    返回：
        balanced_df: 类别平衡后的 DataFrame
        
    示例：
        原始：ham(4825条) + spam(747条) = 5572条
        平衡后：ham(747条) + spam(747条) = 1494条
    """
    label_col = df["Label"]
    
    # 🔍 兼容性处理：标签可能是字符串（'spam'/'ham'）或数字（0/1）
    spam_mask = (label_col == "spam") | (label_col == 1)
    ham_mask = (label_col == "ham") | (label_col == 0)
    
    # 统计垃圾短信数量（少数类）
    spam_count = int(spam_mask.sum())
    
    if spam_count == 0 or ham_mask.sum() == 0:
        raise ValueError("数据集缺少 spam 或 ham 类别，无法平衡")
    
    # 🎲 从多数类（ham）中随机采样，数量与少数类相同
    # random_state=123 确保每次采样结果相同，便于复现
    ham_subset = df[ham_mask].sample(spam_count, random_state=123)
    
    # 📦 合并两个类别的数据
    balanced = pd.concat([ham_subset, df[spam_mask]])
    
    # 🔀 随机打乱数据，避免数据顺序偏差
    return balanced.sample(frac=1, random_state=123).reset_index(drop=True)


def random_split(df, train_frac=0.7, val_frac=0.1):
    """
    随机划分数据集为训练集、验证集和测试集
    
    🎯 目的：将数据分为三部分
    - 训练集：用于模型参数更新
    - 验证集：用于调整超参数和早停
    - 测试集：用于最终性能评估
    
    参数：
        df: pandas DataFrame
        train_frac: 训练集比例（默认 0.7 = 70%）
        val_frac: 验证集比例（默认 0.1 = 10%）
        
    返回：
        train_df, val_df, test_df: 三个数据框
        
    示例：
        总共1494条 → 训练1046条(70%) + 验证149条(10%) + 测试299条(20%)
    """
    # 🔀 随机打乱数据
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)
    
    # 📏 计算划分点
    train_end = int(len(df) * train_frac)
    val_end = train_end + int(len(df) * val_frac)
    
    # ✂️ 执行划分
    train_df = df[:train_end]
    val_df = df[train_end:val_end]
    test_df = df[val_end:]
    
    return train_df, val_df, test_df


def ensure_sms_file(local_path="sms_spam_collection/SMSSpamCollection.tsv"):
    """
    确保 SMS 数据文件存在，如果不存在则自动下载
    
    🎯 目的：自动化数据准备流程
    如果本地已有数据文件，直接返回路径；
    如果没有，则从 UCI 仓库下载并解压。
    
    参数：
        local_path: 本地文件路径（可选）
        
    返回：
        str: 数据文件的实际路径
        
    说明：
        数据文件格式为 TSV（Tab-Separated Values），每行包含标签和文本
    """
    # ✅ 如果文件已存在，直接返回
    if os.path.exists(local_path):
        return local_path
    
    # 📥 从 UCI 机器学习仓库下载数据
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"
    print(f"📥 正在从 UCI 仓库下载数据...")
    print(f"   URL: {url}")
    
    try:
        # 🌐 发送 HTTP 请求下载 ZIP 文件
        resp = requests.get(url, timeout=30)
        resp.raise_for_status()  # 如果请求失败，抛出异常
        
        # 📦 解压 ZIP 文件，读取其中的 SMSSpamCollection 文件
        with zipfile.ZipFile(io.BytesIO(resp.content)) as zf:
            raw = zf.read("SMSSpamCollection").decode("utf-8")
        
        # 💾 保存到本地文件
        out_path = "SMSSpamCollection.tsv"
        with open(out_path, "w", encoding="utf-8") as f:
            f.write(raw)
        
        print(f"✅ 数据下载成功，保存到: {out_path}")
        return out_path
    
    except Exception as e:
        print(f"❌ 数据下载失败: {e}")
        print(f"💡 请手动下载并放置到当前目录")
        raise


def load_sms_dataframe():
    """
    加载并预处理 SMS 数据集
    
    🎯 目的：一站式数据加载函数
    整合了下载、读取、平衡、标签转换和划分的完整流程
    
    返回：
        train_df, val_df, test_df: 三个已处理的 DataFrame
        
    数据处理步骤：
        1. 确保数据文件存在（自动下载）
        2. 读取 TSV 文件
        3. 平衡类别分布
        4. 标签转换（ham→0, spam→1）
        5. 划分训练/验证/测试集
    """
    # 📂 获取数据文件路径（如需要会自动下载）
    data_path = ensure_sms_file()
    
    # 📖 读取 TSV 文件
    # sep="\t" 指定制表符分隔
    # names 指定列名
    # header=None 表示文件没有表头行
    df = pd.read_csv(data_path, sep="\t", names=["Label", "Text"], header=None)
    
    print(f"📊 原始数据加载完成: {len(df)} 条")
    print(f"   - ham (正常): {(df['Label']=='ham').sum()} 条")
    print(f"   - spam (垃圾): {(df['Label']=='spam').sum()} 条")
    
    # ⚖️ 创建类别平衡的数据集
    balanced = create_balanced_dataset(df)
    print(f"⚖️  数据平衡后: {len(balanced)} 条（每类 {len(balanced)//2} 条）")
    
    # 🔄 标签转换：字符串 → 数字
    # 机器学习模型需要数字标签
    label_map = {"ham": 0, "spam": 1}
    balanced["Label"] = balanced["Label"].apply(
        lambda v: label_map[v] if v in label_map else int(v)
    )
    
    # ✂️ 划分数据集
    train_df, val_df, test_df = random_split(balanced, train_frac=0.7, val_frac=0.1)
    print(f"✂️  数据划分完成:")
    print(f"   - 训练集: {len(train_df)} 条")
    print(f"   - 验证集: {len(val_df)} 条")
    print(f"   - 测试集: {len(test_df)} 条")
    
    return train_df, val_df, test_df


# 🎓 初学者提示：
# 以上函数构成了完整的数据处理流水线，每个函数职责单一，便于理解和调试。
# 在实际项目中，建议将数据处理代码模块化，提高代码复用性。
print("✅ 数据处理函数定义完成")


In [ ]:
# 📦 PyTorch Dataset 类定义
# 功能：将 DataFrame 转换为 PyTorch 可用的数据集格式

class SpamSequenceDataset(Dataset):
    """
    SMS 垃圾短信数据集类
    
    🎯 目的：将 pandas DataFrame 转换为 PyTorch Dataset
    这个类继承自 torch.utils.data.Dataset，实现了 PyTorch 数据加载的标准接口
    
    与手动实现的区别：
    - 手动实现：使用 tiktoken 分词器，手动处理 padding
    - PEFT版本：使用 Hugging Face Tokenizer，自动处理 padding 和 attention_mask
    
    关键特性：
    1. 使用 Hugging Face tokenizer，与模型完美配合
    2. 自动生成 attention_mask（标记哪些是真实 token，哪些是 padding）
    3. 批量预处理所有文本，提高效率
    """
    
    def __init__(self, df, tokenizer, max_length=96):
        """
        初始化数据集
        
        参数：
            df: pandas DataFrame，包含 'Text' 和 'Label' 列
            tokenizer: Hugging Face tokenizer 对象
            max_length: 最大序列长度，超过会截断，不足会填充
            
        处理流程：
            1. 批量分词所有文本
            2. 统一长度（截断/填充到 max_length）
            3. 转换为 PyTorch tensors
            4. 保存 labels 为数字张量
        """
        # 🔤 批量分词所有文本
        # tokenizer() 函数的参数说明：
        # - truncation=True: 如果文本超过 max_length，自动截断
        # - padding="max_length": 填充到 max_length 长度
        # - max_length=96: 最大序列长度
        # - return_tensors="pt": 返回 PyTorch tensors
        encodings = tokenizer(
            df["Text"].tolist(),          # 将 pandas Series 转为 list
            truncation=True,               # 启用截断
            padding="max_length",          # 填充到最大长度
            max_length=max_length,         # 最大长度
            return_tensors="pt",           # 返回 PyTorch 张量
        )
        
        # 📊 保存编码结果
        # input_ids: token ID 序列，形状 [N, max_length]
        self.input_ids = encodings["input_ids"]
        
        # attention_mask: 注意力掩码，形状 [N, max_length]
        # 1 表示真实 token，0 表示 padding token
        # 模型会忽略 attention_mask=0 的位置
        self.attention_mask = encodings["attention_mask"]
        
        # 🏷️ 保存标签
        # 转换为 long 类型（int64），这是 PyTorch 交叉熵损失要求的类型
        self.labels = torch.tensor(df["Label"].tolist(), dtype=torch.long)
        
        # 📏 保存数据集大小
        self._len = len(self.labels)
        
        print(f"✅ 数据集创建完成:")
        print(f"   - 样本数: {self._len}")
        print(f"   - 序列长度: {max_length}")
        print(f"   - 输入形状: {self.input_ids.shape}")
    
    def __len__(self):
        """
        返回数据集大小
        
        🎯 目的：告诉 DataLoader 数据集有多少样本
        这是 PyTorch Dataset 必须实现的方法之一
        """
        return self._len
    
    def __getitem__(self, idx):
        """
        获取单个样本
        
        🎯 目的：根据索引返回一个样本
        这是 PyTorch Dataset 必须实现的方法之一
        
        参数：
            idx: 样本索引（0 到 len-1）
            
        返回：
            dict: 包含 input_ids, attention_mask, labels 的字典
            
        说明：
            返回字典格式是 Hugging Face 模型的标准输入格式，
            可以直接使用 model(**batch) 进行解包传参
        """
        return {
            "input_ids": self.input_ids[idx],           # Token ID 序列
            "attention_mask": self.attention_mask[idx], # 注意力掩码
            "labels": self.labels[idx],                 # 标签（0 或 1）
        }


# 🎓 初学者提示：
# Dataset 类只负责数据的"组织"，不负责数据的"批量加载"。
# 批量加载由 DataLoader 完成，它会：
# 1. 调用 __getitem__ 获取多个样本
# 2. 自动将这些样本组合成 batch
# 3. 可选地打乱数据顺序（shuffle=True）
# 4. 可选地使用多进程加速（num_workers>0）

print("✅ Dataset 类定义完成")


# 4️⃣ 执行数据加载

## 🚀 创建 Tokenizer 和加载数据

现在我们将：
1. 加载 GPT-2 tokenizer
2. 加载并处理 SMS 数据集
3. 创建 PyTorch DataLoader


In [ ]:
# 🔤 加载 Tokenizer
# 功能：从 Hugging Face Hub 加载预训练的 GPT-2 分词器

print("🔤 加载 GPT-2 Tokenizer...")

# 📥 从 Hugging Face 下载并加载 GPT-2 tokenizer
# AutoTokenizer.from_pretrained() 会：
# 1. 检查本地缓存（~/.cache/huggingface/）
# 2. 如果没有，从 Hub 下载
# 3. 加载词表和分词规则
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# 🔧 配置 padding token
# GPT-2 原始设计用于生成任务，没有 padding token
# 但分类任务需要将不同长度的文本对齐到相同长度
# 解决方案：将 eos_token（结束符）复用为 pad_token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"   ⚙️  设置 pad_token = eos_token (ID: {tokenizer.eos_token_id})")

print(f"✅ Tokenizer 加载完成")
print(f"   - 词表大小: {tokenizer.vocab_size:,}")
print(f"   - 模型类型: {tokenizer.__class__.__name__}")

# 📊 加载并划分数据
print("\n📊 加载 SMS 数据集...")
train_df, val_df, test_df = load_sms_dataframe()

# 📏 设置最大序列长度
# 说明：这个值需要平衡以下因素：
# - 太短：可能截断重要信息
# - 太长：增加计算成本和显存占用
# - 96 是经过实验的合理值，足以覆盖大部分短信
max_length = 96
print(f"\n📏 设置最大序列长度: {max_length}")

# 🔨 创建 PyTorch Dataset
print("\n🔨 创建 PyTorch Dataset...")
train_dataset = SpamSequenceDataset(train_df, tokenizer, max_length=max_length)
val_dataset = SpamSequenceDataset(val_df, tokenizer, max_length=max_length)
test_dataset = SpamSequenceDataset(test_df, tokenizer, max_length=max_length)

# 📦 创建 DataLoader
# 功能：批量加载数据，支持自动打乱、多进程等
print("\n📦 创建 DataLoader...")

# 🎯 DataLoader 参数说明：
# - batch_size: 每批样本数量
#   - 较大：训练快，但显存占用高
#   - 较小：显存友好，但训练慢
#   - 8 是适中的值
# - shuffle: 是否打乱数据
#   - 训练集：True（避免顺序偏差）
#   - 验证/测试集：False（保持顺序，便于分析）

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

print(f"✅ DataLoader 创建完成")
print(f"   - 训练批次数: {len(train_loader)}")
print(f"   - 验证批次数: {len(val_loader)}")
print(f"   - 测试批次数: {len(test_loader)}")
print(f"   - 每批大小: 8")

# 🔍 数据检查：查看一个样本
print("\n🔍 样本检查:")
sample_batch = next(iter(train_loader))
print(f"   - input_ids 形状: {sample_batch['input_ids'].shape}")
print(f"   - attention_mask 形状: {sample_batch['attention_mask'].shape}")
print(f"   - labels 形状: {sample_batch['labels'].shape}")
print(f"   - 标签分布: {sample_batch['labels'].tolist()}")

print("\n✅ 数据准备完成！")


# 5️⃣ 使用 PEFT 库应用 LoRA

## 🎯 PEFT 核心概念

**PEFT (Parameter-Efficient Fine-Tuning)** 是 Hugging Face 提供的参数高效微调库，支持多种方法：

- **LoRA**: Low-Rank Adaptation（本教程使用）
- **LoHa**: Low-Rank Hadamard Product
- **LoKr**: Low-Rank Kronecker Product
- **Prefix Tuning**: 可学习的前缀
- **P-Tuning**: 提示微调
- **Adapter**: 适配器层

## 🔧 LoRA 配置详解

使用 PEFT 库应用 LoRA 只需三步：
1. 创建 `LoraConfig` 配置对象
2. 加载基础模型
3. 使用 `get_peft_model()` 应用 LoRA

让我们深入了解每个参数的含义。


In [ ]:
# 🤖 加载基础模型并应用 LoRA
# 功能：这是 PEFT 库使用的核心步骤

print("🤖 步骤 1: 加载基础模型...")
print("=" * 60)

# 📥 从 Hugging Face Hub 加载 GPT-2 分类模型
# AutoModelForSequenceClassification 会：
# 1. 加载 GPT-2 的预训练权重
# 2. 在顶部添加一个分类头（线性层）
# 3. num_labels=2 表示二分类任务
base_model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",           # 模型名称
    num_labels=2      # 分类类别数
)

# 🔧 配置模型的 padding token
# 必须与 tokenizer 的配置保持一致
base_model.config.pad_token_id = tokenizer.pad_token_id

# 📏 调整 embedding 层大小
# GPT-2 的词表大小是 50257，如果 tokenizer 添加了新 token，需要调整
base_model.resize_token_embeddings(len(tokenizer))

print(f"✅ 基础模型加载完成")
print(f"   - 模型类型: {base_model.__class__.__name__}")
print(f"   - 分类类别: {base_model.config.num_labels}")
print(f"   - 词表大小: {base_model.config.vocab_size:,}")

# 🔧 步骤 2: 创建 LoRA 配置
print("\n🔧 步骤 2: 创建 LoRA 配置...")
print("=" * 60)

# 📋 LoRA 配置参数详解
lora_config = LoraConfig(
    # 🎯 task_type: 任务类型
    # - TaskType.SEQ_CLS: 序列分类任务
    # - TaskType.CAUSAL_LM: 因果语言模型（生成任务）
    # - TaskType.SEQ_2_SEQ_LM: 序列到序列任务
    # 不同任务类型会影响 LoRA 层的插入位置
    task_type=TaskType.SEQ_CLS,
    
    # 📊 r: LoRA 的秩（rank）
    # 这是 LoRA 最核心的超参数！
    # - 控制 A 和 B 矩阵的中间维度
    # - 较小的值（如 4-8）：参数少，训练快，但表达能力弱
    # - 较大的值（如 16-32）：参数多，表达能力强，但接近全量微调
    # - 推荐起始值：8
    # - 本教程使用：8（平衡效率和性能）
    r=8,
    
    # 🎚️ lora_alpha: LoRA 缩放因子
    # - 控制 LoRA 输出的影响强度
    # - 常见设置：alpha = 2×r 或 alpha = r
    # - 本教程：alpha = 2×r = 16
    # - 数学公式：LoRA 输出会乘以 (alpha/r) 的缩放系数
    lora_alpha=16,
    
    # 🎲 lora_dropout: Dropout 比率
    # - 在 LoRA 层中应用 dropout 正则化
    # - 范围：0.0（不使用）到 0.2（较强正则化）
    # - 作用：防止过拟合，提高泛化能力
    # - 本教程：0.05（轻微正则化）
    lora_dropout=0.05,
    
    # 🎯 target_modules: 要替换的模块名称列表
    # 这是 PEFT 库的强大功能：选择性地只对部分层应用 LoRA
    # GPT-2 的模块名称：
    # - "c_attn": 注意力层的 QKV 投影（最重要）
    # - "c_proj": 注意力层的输出投影
    # - "c_fc": 前馈网络的第一层
    # - "c_proj": 前馈网络的第二层（与注意力同名，PEFT 会区分）
    # 
    # 💡 选择策略：
    # - 只替换 ["c_attn"]: 最少参数，适合显存极小场景
    # - 替换 ["c_attn", "c_proj"]: 平衡选择
    # - 替换 ["c_attn", "c_fc", "c_proj"]: 本教程使用，覆盖主要层
    # - 不建议替换 embedding 层和分类头
    target_modules=["c_attn", "c_fc", "c_proj"],
    
    # ⚙️ 其他可选参数（使用默认值）：
    # - bias="none": 不训练 bias 参数
    # - fan_in_fan_out=False: 权重矩阵的组织方式
    # - modules_to_save=None: 除 LoRA 外需要训练的模块
)

print("✅ LoRA 配置创建完成:")
print(f"   - 秩 (r): {lora_config.r}")
print(f"   - 缩放因子 (alpha): {lora_config.lora_alpha}")
print(f"   - Dropout: {lora_config.lora_dropout}")
print(f"   - 目标模块: {lora_config.target_modules}")

# 🚀 步骤 3: 应用 LoRA 到模型
print("\n🚀 步骤 3: 应用 LoRA...")
print("=" * 60)

# 🔑 核心函数：get_peft_model()
# 功能：将 LoRA 配置应用到基础模型
# 工作原理：
# 1. 遍历模型的所有模块
# 2. 找到 target_modules 中指定的线性层
# 3. 用 LoRA 增强版本替换这些层
# 4. 冻结原始参数，只让 LoRA 参数可训练
model = get_peft_model(base_model, lora_config)

print("✅ LoRA 应用完成！")

# 📊 打印参数统计
# print_trainable_parameters() 是 PEFT 模型的便捷方法
# 会显示可训练参数数量和占比
print("\n📊 参数统计:")
model.print_trainable_parameters()

# 🎮 将模型移动到 GPU/CPU
model.to(device)
print(f"\n🎮 模型已移动到设备: {device}")

# 🔍 模型结构检查（可选）
print("\n🔍 模型结构预览:")
print(f"   - 模型类型: {type(model).__name__}")
print(f"   - 基础模型: {type(model.base_model).__name__}")
print(f"   - LoRA 配置: {model.peft_config}")

print("\n✅ 模型准备完成！")


# 6️⃣ 训练和评估函数

## 🎯 训练流程概述

标准的深度学习训练流程包括：
1. **训练循环**：前向传播 → 计算损失 → 反向传播 → 更新参数
2. **评估循环**：在验证集上计算性能指标
3. **学习率调度**：动态调整学习率
4. **早停机制**：防止过拟合

## 📊 PEFT 模型的训练特点

使用 PEFT 后，训练代码与普通 PyTorch 模型**完全相同**！
- ✅ 自动只优化 LoRA 参数
- ✅ 自动冻结原始参数
- ✅ 无需手动筛选参数


In [ ]:
# 📊 评估函数定义
# 功能：计算模型在数据集上的准确率和损失

def evaluate(model, data_loader):
    """
    评估模型性能
    
    🎯 目的：在验证集或测试集上计算准确率和平均损失
    
    参数：
        model: 待评估的模型
        data_loader: DataLoader 对象（验证集或测试集）
        
    返回：
        acc: 准确率 (0-1 之间)
        avg_loss: 平均损失值
        
    工作流程：
        1. 切换到评估模式 (model.eval())
        2. 禁用梯度计算 (torch.no_grad())
        3. 遍历所有批次计算损失和准确率
        4. 返回平均值
    """
    # 🔧 切换到评估模式
    # 作用：
    # 1. 禁用 Dropout（所有神经元都参与计算）
    # 2. 禁用 Batch Normalization 的更新（使用训练时的统计量）
    # 3. 某些层的行为会改变
    model.eval()
    
    # 📊 初始化统计变量
    total_loss = 0.0    # 累积损失
    correct = 0         # 正确预测数
    total = 0           # 总样本数
    
    # 🚫 禁用梯度计算
    # 作用：节省显存，加快计算速度
    # 评估时不需要梯度，因为不进行参数更新
    with torch.no_grad():
        # 🔄 遍历数据批次
        for batch in data_loader:
            # 🎮 将数据移动到设备（GPU/CPU）
            # 字典推导式：将 batch 中的每个张量都移动到目标设备
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # 🔮 前向传播
            # model(**batch) 等价于 model(input_ids=..., attention_mask=..., labels=...)
            # Hugging Face 模型返回一个包含 loss 和 logits 的对象
            outputs = model(**batch)
            
            # 📉 提取损失值
            # 当传入 labels 时，模型会自动计算交叉熵损失
            loss = outputs.loss
            
            # 🎯 获取预测结果
            # logits: [batch_size, num_labels] 形状的张量
            # argmax(dim=-1): 沿最后一维取最大值的索引，得到预测类别
            preds = outputs.logits.argmax(dim=-1)
            
            # 📊 累积统计量
            total_loss += loss.item()  # 累加损失（转为 Python 数值）
            correct += (preds == batch["labels"]).sum().item()  # 统计正确预测数
            total += batch["labels"].size(0)  # 累加样本数
    
    # 📐 计算平均值
    avg_loss = total_loss / max(len(data_loader), 1)  # 平均损失
    acc = correct / total if total > 0 else 0.0        # 准确率
    
    return acc, avg_loss


# 🚀 训练函数定义
# 功能：执行完整的训练流程

def train(model, train_loader, val_loader, epochs=3, lr=5e-4):
    """
    训练 LoRA 模型
    
    🎯 目的：执行完整的训练循环，包括优化器、学习率调度和梯度裁剪
    
    参数：
        model: PEFT 模型（已应用 LoRA）
        train_loader: 训练数据加载器
        val_loader: 验证数据加载器
        epochs: 训练轮数（默认 3）
        lr: 学习率（默认 5e-4）
        
    训练流程：
        每个 epoch:
            1. 训练阶段：遍历训练集，更新参数
            2. 评估阶段：在验证集上计算性能
            3. 学习率调度：根据验证损失调整学习率
    
    💡 LoRA 微调的学习率建议：
    - 全量微调：1e-5 ~ 5e-5（小学习率）
    - LoRA 微调：5e-4 ~ 1e-3（可以用较大学习率）
    """
    print("🚀 开始训练...")
    print("=" * 60)
    
    # ⚙️ 创建优化器
    # AdamW: Adam 优化器的改进版，带权重衰减（Weight Decay）
    # model.parameters() 会自动只返回 requires_grad=True 的参数
    # 因此这里会自动只优化 LoRA 参数！
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    
    print(f"⚙️  优化器: AdamW (lr={lr})")
    
    # 📅 创建学习率调度器
    # get_linear_schedule_with_warmup: 带预热的线性学习率调度
    # 
    # 🔥 Warmup（预热）的作用：
    # - 训练初期使用较小的学习率，逐渐增加到目标学习率
    # - 防止训练初期梯度过大导致的不稳定
    # - 公式：lr = base_lr * (current_step / warmup_steps)
    # 
    # 📉 线性衰减：
    # - Warmup 结束后，学习率线性降低到 0
    # - 有助于训练后期的精细调整
    total_steps = epochs * len(train_loader)  # 总训练步数
    warmup_steps = max(10, int(0.1 * total_steps))  # Warmup 步数（10% 的总步数）
    
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps,
    )
    
    print(f"📅 学习率调度器: Linear with Warmup")
    print(f"   - 总步数: {total_steps}")
    print(f"   - Warmup 步数: {warmup_steps}")
    
    # 🔄 训练循环
    for epoch in range(epochs):
        # 📚 训练阶段
        model.train()  # 切换到训练模式
        running_loss = 0.0
        
        # 📊 使用 tqdm 显示进度条
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        
        for batch in pbar:
            # 🎮 数据移动到设备
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # 🧹 清空梯度
            # PyTorch 默认累积梯度，必须手动清零
            optimizer.zero_grad()
            
            # 🔮 前向传播
            outputs = model(**batch)
            loss = outputs.loss
            
            # 🔄 反向传播
            # 计算所有参数的梯度
            loss.backward()
            
            # ✂️ 梯度裁剪
            # 限制梯度范数，防止梯度爆炸
            # max_norm=1.0: 如果梯度范数 > 1.0，按比例缩放到 1.0
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # ⚡ 参数更新
            optimizer.step()      # 使用梯度更新参数
            scheduler.step()      # 更新学习率
            
            # 📊 累积损失
            running_loss += loss.item()
            
            # 📈 更新进度条显示
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        # 📊 计算本轮平均损失
        avg_train_loss = running_loss / len(train_loader)
        
        # 📈 验证阶段
        val_acc, val_loss = evaluate(model, val_loader)
        
        # 📢 打印结果
        print(f"Epoch {epoch+1}/{epochs}:")
        print(f"   - 训练损失: {avg_train_loss:.4f}")
        print(f"   - 验证损失: {val_loss:.4f}")
        print(f"   - 验证准确率: {val_acc*100:.1f}%")
        print(f"   - 当前学习率: {scheduler.get_last_lr()[0]:.2e}")
    
    print("\n✅ 训练完成！")


# 🎓 初学者提示：
# 以上训练函数集成了多个训练技巧：
# 1. AdamW 优化器：比 SGD 收敛更快，比 Adam 泛化更好
# 2. 学习率预热：训练初期稳定性更好
# 3. 梯度裁剪：防止梯度爆炸
# 4. 进度条显示：实时监控训练状态
# 
# 这些都是工业界的标准做法，建议在自己的项目中使用。

print("✅ 训练和评估函数定义完成")


In [ ]:
# 🚀 执行训练
# 功能：调用训练函数开始微调

# ⚙️ 训练参数设置
EPOCHS = 3          # 训练轮数
LEARNING_RATE = 5e-4  # 学习率

print("⚙️  训练参数:")
print(f"   - 训练轮数: {EPOCHS}")
print(f"   - 学习率: {LEARNING_RATE}")
print(f"   - 批次大小: 8")
print(f"   - 优化器: AdamW")
print(f"   - 学习率调度: Linear with Warmup")
print("")

# 🚀 开始训练
train(model, train_loader, val_loader, epochs=EPOCHS, lr=LEARNING_RATE)

# 📊 在测试集上评估
print("\n📊 最终测试...")
print("=" * 60)
test_acc, test_loss = evaluate(model, test_loader)

print(f"✅ 测试结果:")
print(f"   - 测试损失: {test_loss:.4f}")
print(f"   - 测试准确率: {test_acc*100:.2f}%")

# 🎯 性能分析
if test_acc >= 0.95:
    print("\n🎉 优秀！模型达到了 95%+ 的准确率！")
elif test_acc >= 0.90:
    print("\n👍 很好！模型达到了 90%+ 的准确率！")
elif test_acc >= 0.85:
    print("\n✅ 不错！模型达到了 85%+ 的准确率！")
else:
    print("\n💡 提示：准确率偏低，建议调整超参数或增加训练轮数")


# 8️⃣ 实际应用：文本分类推理

## 🎯 使用训练好的模型进行预测

现在让我们使用训练好的模型对新文本进行分类，看看效果如何。


In [ ]:
# 🔮 推理演示
# 功能：使用训练好的模型对新文本进行分类预测

# 📝 准备测试样本
# 包含明显的正常短信和垃圾短信
sample_texts = [
    "Hey, want to grab lunch together?",                           # 正常短信
    "URGENT! You've won $5000! Click now to claim your prize!",   # 垃圾短信
    "The meeting has been moved to 2pm",                          # 正常短信
    "Free iPhone! Limited time offer! Call immediately!",         # 垃圾短信
    "Thanks for your help yesterday",                              # 正常短信
    "Congratulations! You are selected as winner. Text WIN now!",  # 垃圾短信
]

print("🔮 推理演示")
print("=" * 60)
print(f"测试 {len(sample_texts)} 条样本...\n")

# 🔧 切换到评估模式
model.eval()

# 🔄 遍历每个测试样本
for i, text in enumerate(sample_texts, 1):
    # 🔤 使用 tokenizer 编码文本
    # 参数说明：
    # - return_tensors="pt": 返回 PyTorch 张量
    # - truncation=True: 如果超长则截断
    # - padding="max_length": 填充到最大长度
    # - max_length: 与训练时保持一致
    encoded = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=max_length,
    )
    
    # 🎮 将数据移动到设备
    encoded = {k: v.to(device) for k, v in encoded.items()}
    
    # 🚫 禁用梯度计算（推理时不需要）
    with torch.no_grad():
        # 🔮 前向传播获取 logits
        logits = model(**encoded).logits
        
        # 📊 计算概率分布
        # softmax 将 logits 转换为概率（和为 1）
        probs = torch.softmax(logits, dim=-1)[0]
    
    # 🎯 获取预测结果
    pred = torch.argmax(probs).item()  # 预测类别（0 或 1）
    label = "spam" if pred == 1 else "ham"  # 转换为文本标签
    
    # 📊 提取概率值
    ham_prob = probs[0].item()   # 正常短信概率
    spam_prob = probs[1].item()  # 垃圾短信概率
    
    # 📢 打印结果
    print(f"【样本 {i}】")
    print(f"文本: {text[:60]}{'...' if len(text) > 60 else ''}")
    print(f"预测: {label.upper()}")
    print(f"置信度: ham={ham_prob:.3f}, spam={spam_prob:.3f}")
    
    # 🎨 添加表情符号增强可读性
    if pred == 1:
        print("🚨 分类结果：垃圾短信")
    else:
        print("✅ 分类结果：正常短信")
    
    print()

print("✅ 推理演示完成！")


# 9️⃣ 模型保存和加载

## 💾 PEFT 模型的保存策略

PEFT 库提供了非常方便的模型保存功能：

### 🎯 两种保存方式

1. **只保存 LoRA 参数**（推荐）
   - 文件很小（几 MB）
   - 需要配合原始模型使用
   - 适合模型分发和多任务场景

2. **保存合并后的完整模型**
   - 将 LoRA 参数合并到原始权重
   - 文件较大
   - 可以作为独立模型使用


In [ ]:
# 💾 方式 1：只保存 LoRA 参数（推荐）
# 功能：保存轻量级的 LoRA 适配器文件

print("💾 保存 LoRA 参数...")

# 📁 指定保存路径
lora_save_path = "./lora_sms_spam_classifier"

# 💾 保存 LoRA 适配器
# save_pretrained() 会保存：
# 1. adapter_config.json: LoRA 配置
# 2. adapter_model.safetensors: LoRA 权重（推荐格式）
# 或 adapter_model.bin: LoRA 权重（传统格式）
model.save_pretrained(lora_save_path)

print(f"✅ LoRA 参数已保存到: {lora_save_path}")

# 📊 检查保存的文件
import os
saved_files = os.listdir(lora_save_path)
print(f"📂 保存的文件: {saved_files}")

# 📏 检查文件大小
total_size = sum(os.path.getsize(os.path.join(lora_save_path, f)) for f in saved_files)
print(f"📊 总大小: {total_size / (1024*1024):.2f} MB")

# 💡 加载方式示例（代码仅供参考，不实际执行）
print("\n💡 加载 LoRA 模型的方法:")
print("```python")
print("from peft import PeftModel")
print("from transformers import AutoModelForSequenceClassification")
print("")
print("# 1. 加载基础模型")
print('base_model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2)')
print("")
print("# 2. 加载 LoRA 适配器")
print('model = PeftModel.from_pretrained(base_model, "./lora_sms_spam_classifier")')
print("")
print("# 3. 使用模型进行推理")
print("model.eval()")
print("# ... 推理代码 ...")
print("```")

print("\n" + "="*60)

# 🔗 方式 2：合并并保存完整模型（可选）
print("\n🔗 方式 2：合并 LoRA 参数到基础模型")

# 📝 合并说明
print("💡 合并操作会将 LoRA 参数加到原始权重上：")
print("   W_new = W_original + α/r * (A @ B)")
print("")
print("⚠️  注意：合并后将失去切换 LoRA 适配器的能力")
print("")

# 🔧 合并模型（示例代码）
print("🔧 合并代码示例（不执行，仅展示）:")
print("```python")
print("# 合并 LoRA 参数到基础模型")
print("merged_model = model.merge_and_unload()")
print("")
print("# 保存完整模型")
print('merged_model.save_pretrained("./merged_sms_spam_classifier")')
print('tokenizer.save_pretrained("./merged_sms_spam_classifier")')
print("")
print("# 加载完整模型")
print('loaded_model = AutoModelForSequenceClassification.from_pretrained(')
print('    "./merged_sms_spam_classifier"')
print(')')
print("```")

print("\n✅ 模型保存完成！")

# 📊 保存方式对比
print("\n📊 两种保存方式对比:")
print("┌─────────────────┬──────────────┬──────────────┐")
print("│      特性       │  LoRA 参数   │  完整模型    │")
print("├─────────────────┼──────────────┼──────────────┤")
print("│  文件大小       │  几 MB       │  数百 MB     │")
print("│  加载速度       │  快          │  较慢        │")
print("│  多任务切换     │  支持        │  不支持      │")
print("│  独立使用       │  需基础模型  │  可独立使用  │")
print("│  推荐场景       │  生产环境    │  演示/部署   │")
print("└─────────────────┴──────────────┴──────────────┘")


# 🎓 教程总结

## ✨ 核心要点

### PEFT 库使用三步曲
1. 创建 LoRAConfig
2. 加载基础模型  
3. 调用 get_peft_model()

### 关键优势
- 代码简洁（~10行 vs ~200行）
- 工业级实现
- 官方维护更新
- 与 Transformers 无缝集成

## 🎯 超参数建议

- **rank**: 8-16（推荐起点）
- **alpha**: 2 × rank
- **dropout**: 0.05-0.1
- **learning_rate**: 5e-4 到 1e-3

## 💡 最佳实践

1. 使用学习率预热（Warmup）
2. 添加梯度裁剪
3. 监控验证集性能
4. 保存轻量级 LoRA 参数

## 🚀 进阶方向

- QLoRA：结合量化
- 多任务 LoRA
- 大模型微调（LLaMA、Mistral）

🎉 恭喜完成学习！
